In [1]:
from sqlalchemy import select
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column
from sqlalchemy.types import Text

In [2]:
class Base(DeclarativeBase):
    pass

class Message(Base):
    __tablename__ = "messages"

    id: Mapped[int] = mapped_column(primary_key=True)
    content: Mapped[str] = mapped_column(Text())

    def __repr__(self):
        return f"Message(id={self.id}, content={self.content})"

テーブル作れる。

In [3]:
url = "sqlite+aiosqlite:///example.db"

engine = create_async_engine(url=url, echo=True, echo_pool=True)
async with engine.begin() as conn:
    await conn.run_sync(Base.metadata.drop_all)
    await conn.run_sync(Base.metadata.create_all)

2024-03-14 20:50:18,143 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-14 20:50:18,144 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("messages")
2024-03-14 20:50:18,145 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-14 20:50:18,147 INFO sqlalchemy.engine.Engine 
DROP TABLE messages
2024-03-14 20:50:18,147 INFO sqlalchemy.engine.Engine [no key 0.00051s] ()
2024-03-14 20:50:18,150 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("messages")
2024-03-14 20:50:18,150 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-14 20:50:18,152 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("messages")
2024-03-14 20:50:18,153 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-14 20:50:18,155 INFO sqlalchemy.engine.Engine 
CREATE TABLE messages (
	id INTEGER NOT NULL, 
	content TEXT NOT NULL, 
	PRIMARY KEY (id)
)


2024-03-14 20:50:18,155 INFO sqlalchemy.engine.Engine [no key 0.00069s] ()
2024-03-14 20:50:18,158 INFO sqlalchemy.engine.Engine COMMIT


`add`して`commit`で`INSERT`。

In [4]:
session = AsyncSession(engine)
try:
    for content in ["foo", "bar", "baz", "qux"]:
        session.add(Message(content=content))
    await session.commit()
finally:
    await session.close()

2024-03-14 20:50:18,168 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-14 20:50:18,170 INFO sqlalchemy.engine.Engine INSERT INTO messages (content) VALUES (?) RETURNING id
2024-03-14 20:50:18,170 INFO sqlalchemy.engine.Engine [generated in 0.00010s (insertmanyvalues) 1/4 (ordered; batch not supported)] ('foo',)
2024-03-14 20:50:18,172 INFO sqlalchemy.engine.Engine INSERT INTO messages (content) VALUES (?) RETURNING id
2024-03-14 20:50:18,173 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/4 (ordered; batch not supported)] ('bar',)
2024-03-14 20:50:18,174 INFO sqlalchemy.engine.Engine INSERT INTO messages (content) VALUES (?) RETURNING id
2024-03-14 20:50:18,175 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/4 (ordered; batch not supported)] ('baz',)
2024-03-14 20:50:18,176 INFO sqlalchemy.engine.Engine INSERT INTO messages (content) VALUES (?) RETURNING id
2024-03-14 20:50:18,177 INFO sqlalchemy.engine.Engine [insertmanyvalues 4/4 (ordered; batch not supported)] ('qux',)
2

個々の条件を`()`で括って`&`で`AND`。

In [5]:
session = AsyncSession(engine)
try:
    result = await session.execute(select(Message).where((1 < Message.id) & (Message.id < 4)))
    for m in result.scalars():
        print(m)
finally:
    await session.close()

2024-03-14 20:50:18,189 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-14 20:50:18,191 INFO sqlalchemy.engine.Engine SELECT messages.id, messages.content 
FROM messages 
WHERE messages.id > ? AND messages.id < ?
2024-03-14 20:50:18,192 INFO sqlalchemy.engine.Engine [generated in 0.00084s] (1, 4)
Message(id=2, content=bar)
Message(id=3, content=baz)
2024-03-14 20:50:18,194 INFO sqlalchemy.engine.Engine ROLLBACK


`|`で`OR`。

In [6]:
session = AsyncSession(engine)
try:
    result = await session.execute(select(Message).where((2 > Message.id) | (Message.id > 3)))
    for m in result.scalars():
        print(m)
finally:
    await session.close()

2024-03-14 20:50:18,203 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-14 20:50:18,204 INFO sqlalchemy.engine.Engine SELECT messages.id, messages.content 
FROM messages 
WHERE messages.id < ? OR messages.id > ?
2024-03-14 20:50:18,205 INFO sqlalchemy.engine.Engine [generated in 0.00056s] (2, 3)
Message(id=1, content=foo)
Message(id=4, content=qux)
2024-03-14 20:50:18,207 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
await engine.dispose()

2024-03-14 20:50:18,214 INFO sqlalchemy.pool.impl.NullPool Pool recreating
